<a href="https://colab.research.google.com/github/fellowship/deep-and-wide-bandit/blob/dev/EDA_LargeSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers

# Mount Drive

In [2]:
from google.colab import drive
drive.mount("/content/GDrive")

Drive already mounted at /content/GDrive; to attempt to forcibly remount, call drive.mount("/content/GDrive", force_remount=True).


In [3]:
import os
import pandas as pd
from zipfile import ZipFile
from transformers import BertTokenizer, BertModel
import re
import torch

#Formatting .describe() calls s.t. floats are displayed in non-scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [4]:
os.chdir(os.getcwd())
zip_path_l = ["/content/GDrive/MyDrive/July_Dec_2019.zip",
               "/content/GDrive/MyDrive/Jan_June_2020.zip", 
               "/content/GDrive/MyDrive/Jul_Sep_2020.zip"]

folder_name_l = [os.path.split(zip_path)[-1][:-4] for zip_path in zip_path_l]

for (zip_path, folder_name) in zip(zip_path_l, folder_name_l):
  
  if not(os.path.exists(folder_name)):
    
    with ZipFile(zip_path, 'r') as zip_obj:

      zip_obj.extractall(folder_name)

In [5]:
file_name_l = ['/content/July_Dec_2019/sends_july_dec_2019_2000',
               '/content/Jan_June_2020/sends_jan_june_2020_2000',
               '/content/Jul_Sep_2020/sends_july_sept_2020_2000']

col_names = ['riid','send_dt','launch_id','sub_sourcedev','opened','unsub',
             'rev_3dv2','aq_dt','aq_mo','aq_dow','aq_period','sends_since_last_open',
             'wk1_opens','wk4_opens','mo3_opens','mo6_opens','yr1_opens',
             'wk1_clicks','wk4_clicks','mo3_clicks','mo6_clicks',
             'rev_dec','rev_raw','snds_dec','snds_opens_dec','wk1_vis',
             'wk4_vis','mo3_vis','mo6_vis','wk1_pv','wk4_pv','mo3_pv','mo6_pv',
             'wk1_secs','wk4_secs','mo3_secs','mo6_secs','prev_optouts',
             'last_order','last_click','last_cart','last_visit','last_ded_open',
             'sent_time','offer_signature_id','dynamic_content_signature_id',
             'message_size','open_cnt','first_open_time','last_open_time',
             'hrs_to_first_open','hrs_to_last_open','click_cnt','first_click_time',
             'last_click_time','offer_category','hrs_to_first_click','hrs_to_last_click',
             'opt_out_cnt','first_opt_out_time','last_opt_out_time','opt_out_source',
             'opt_out_reason','hrs_to_first_opt_out','hrs_to_last_opt_out','campaign_id',
             'subject','marketing_strategy','campaign_type']

#os.remove('/content/July_Dec_2019/sends_july_dec_2019_2000')
#df2_jan_jun_20 = pd.read_csv(file_name_l[1], sep='\t', header=None, names=col_names)
#os.remove('/content/Jan_June_2020/sends_jan_june_2020_2000')
#df3_jul_sep_20 = pd.read_csv(file_name_l[2], sep='\t', header=None, names=col_names)
#os.remove('/content/Jul_Sep_2020/sends_july_sept_2020_2000')

In [6]:
#Inspect the first 1e5 rows of one of the dataframes
df = pd.read_csv(file_name_l[0], sep="\t", header=None, names=col_names, nrows=1e5)
df.head(10)

,riid,send_dt,launch_id,sub_sourcedev,opened,unsub,rev_3dv2,aq_dt,aq_mo,aq_dow,aq_period,sends_since_last_open,wk1_opens,wk4_opens,mo3_opens,mo6_opens,yr1_opens,wk1_clicks,wk4_clicks,mo3_clicks,mo6_clicks,rev_dec,rev_raw,snds_dec,snds_opens_dec,wk1_vis,wk4_vis,mo3_vis,mo6_vis,wk1_pv,wk4_pv,mo3_pv,mo6_pv,wk1_secs,wk4_secs,mo3_secs,mo6_secs,prev_optouts,last_order,last_click,last_cart,last_visit,last_ded_open,sent_time,offer_signature_id,dynamic_content_signature_id,message_size,open_cnt,first_open_time,last_open_time,hrs_to_first_open,hrs_to_last_open,click_cnt,first_click_time,last_click_time,offer_category,hrs_to_first_click,hrs_to_last_click,opt_out_cnt,first_opt_out_time,last_opt_out_time,opt_out_source,opt_out_reason,hrs_to_first_opt_out,hrs_to_last_opt_out,campaign_id,subject,marketing_strategy,campaign_type
0,251392782,2019-08-11,175407402,Checkout_Online,1,0,48.98000,379,7,6,Non-Holiday,16,0,0,1,4,9,0,0,0,1,0.19021,128.97000,6.49108,0.22946,0,0,0,0,0,0,0,0,0,0,0,0,0,378.00000,175.00000,nan,nan,35.00000,2019-08-11 17:11:15,581429320,nan,149211,1.00000,2019-08-11 19:34:58,2019-08-11 19:34:58,2.00000,2.00000,1.00000,2019-08-11 19:35:17,2019-08-11 19:35:17,Mens_Pants_Straight,2.00000,2.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59123842,It’s almost the end,Dockers_Adhoc_Promotional,20190811_US_Dockers_Tops_GraphicTees_30off_FS_...
1,273890682,2019-12-26,176031042,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-14 19:43:46,-1610606741,224451348.00000,111097,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59253142,"<#include ""cms://messagelibrary/email/59253142...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT4_Trigger_20191226
2,273890682,2019-12-26,176031082,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-12 18:43:46,1329572970,746908335.00000,175946,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59253042,"<#include ""cms://messagelibrary/email/59253042...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT3_Trigger_20191226
3,273890682,2019-12-26,176031062,Footer_Desktop,1,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-07 18:20:54,-2128439724,746908335.00000,163449,2.00000,2020-03-07 19:54:09,2020-03-07 20:47:41,1.00000,2.00000,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59251982,"<#include ""cms://messagelibrary/email/59251982...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT1_Trigger_20191226
4,273890682,2019-12-26,176031022,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-10 17:43:46,2066955832,746908335.00000,228854,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59252822,"<#include ""cms://messagelibrary/email/59252822...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT2_Trigger_20191226
5,260343962,2019-11-29,175893902,Lightbox_Mobile,1,0,59.98000,355,12,0,Non-Holiday,53,0,0,0,0,1,0,0,0,0,nan,nan,1.46535,0.01291,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2019-11-29 13:07:15,230762889,nan,163400,2.00000,2019-11-29 16:30:49,2019-11-29 17:01:18,3.00000,4.00000,2.00000,2019-11-29 16:31:06,2019-11-29 17:01:35,"Mens, Sale",3.00000,4.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59216502,50% off absolutely everything + free shipping,Dockers_Adhoc_Promotional,20191129_US_Dockers_Dedicated_BF_50Off_FSFR_AM...
6,206326602,2019-11-29,175893902,My Account_Desktop,1,0,39.00000,1253,6,5,Non-Holiday,0,1,0,0,3,3,0,0,0,1,nan,nan,7.06333,1.05217,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,160.00000,nan,nan,194.00000,2019-11-29 13:08:27,230762889,nan,164971,1.00000,2019-11-29 16:42:59,2019-11-29 16:42:59,3.00000,3.00000,3.00000,2019-11-29 16:43:02,2019-11-29 18:13:32,"Promotion, Mens_ShirtTops, Brand",3.00000,5.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59216502

# Finalizing the data preprocessing

First, we work on this small data snippet and finalise the steps required for cleaning/preprocessing. 

Then, we iterate over each chunk, chain the preprocessing steps to each chunk and keep appending them to an accumulating dataframe.

**QUESTION:** How do we split the train-valid-test set?

1.   For each of the provided files, we employ a 80-20 split.
2.   For the full set of files, we employ a 80-20 split. i.e., the full first 2 files and a portion of the first half of the last file is used for training and remaining is used for validation.



In [7]:
#Too many questions around what to keep, what to drop...
#Hence, currently default behaviour of drop
#Using a control variable dictionary to modify behaviour for relevant column groups
control_variable = {'marketing': False, 'subject':False, 'campaign_type':False,
                    'aq': True,'site': True,'last':True, 'promo_ids': True,
                    'email_cnts':True, 'email_times':True, 'opt_out':True, 
                    'offer_cat':True}

## Drop Cols + Add Scores

In [8]:
"""
Drop less-frequent AND/OR redundant columns
all team members voted to eliminate
"""

#Columns that are finalised to be dropped
df_sample = df.copy()
col_drop_names = ["send_dt", "sub_sourcedev", "rev_dec", "rev_raw", "snds_dec", "snds_opens_dec", 
            "dynamic_content_signature_id"]

df_sample.drop(columns=col_drop_names, inplace=True)
del col_drop_names

"""
Compute the composite recency, frequency (dropped) & retention scores
Dropping the constituent base columns

NOTE 
----
1) Modified recency score to incorporate (1) click data and (2) yr_1 opens
2) Both recency and frequency are strongly correlated
3) sends_since_last_open used for retention score AS WELL AS reward
"""

#Retention Score
df_sample['retention_score'] = df_sample.sends_since_last_open.apply(lambda x: 28/max(1,x)) 

#Recency Score
df_sample["recency_score"] = 16/7*(df_sample['wk1_opens'] + df_sample['wk1_clicks']) 
+ 8/28*(df_sample['wk4_opens'] + df_sample['wk4_clicks']) 
+ 4/64*(df_sample['mo3_opens'] + df_sample['mo3_clicks']) 
+ 2/92*(df_sample['mo6_opens'] + df_sample['mo6_clicks']) 
+ 1/184*df_sample['yr1_opens']

#Frequency Score
"""
Frequency Score is highly correlated with Recency Score. Hence, dropping it.
"""
#df_sample["frequency_score"] = df_sample['wk1_opens'] + df_sample['wk4_opens'] + \
#          df_sample['mo3_opens'] + df_sample['mo6_opens']

#Drop the base columns
col_drop_scores = ["wk1_opens", "wk4_opens", "mo3_opens","mo6_opens","yr1_opens",
                   "wk1_clicks", "wk4_clicks", "mo3_clicks", "mo6_clicks"]

df_sample.drop(columns=col_drop_scores, inplace=True)
del col_drop_scores

"""
Convert sent_time column to datetime object
"""
datetimes = pd.to_datetime(df_sample.sent_time)
df_sample.drop(columns=["sent_time"], inplace=True)
df_sample["sent_time"] = datetimes
del datetimes

## marketing_strategy

Details for an 1e5 chunk:

Dockers_Adhoc_Promotional           34258

Dockers_Program_WelcomeGeneral      21174

Dockers_Adhoc_PromotionalFirst      17053

Dockers_Adhoc_PromotionalLastDay     9419

Dockers_Adhoc_Sale                   7669

Dockers_Adhoc_BrandPlatform          4512

Dockers_Adhoc_Product                1403

Dockers_Program_Product               744

Dockers_Program_Promotional           623

Dockers_Program_ChurnMitigation       353

Name: marketing_strategy, dtype: int64 (?)

In [9]:
"""
Column Name. - marketing_strategy

NOTE
----
1) AdHoc (1) or Program (0)? Might be useful to keep track of this
2) Whether Promo or not is useful to check perhaps
3) Whether Sale or not is useful to check perhaps
"""

if not(control_variable["marketing"]):
  #Create a new column for AdHoc OR Program: Seems to be the type of email
  df_sample["email_type"] = df_sample.marketing_strategy.str.contains("_Adhoc_").fillna(0).astype(int) #1 for AdHoc, 0 for Program

  #Create columns for Promotional and Sale: Other values do not seem useful - Aleksey to get back about correlations
  df_sample["promo"] = df_sample.marketing_strategy.str.contains("Promotional").fillna(0).astype(int)
  df_sample["sale"] = df_sample.marketing_strategy.str.contains("Sale").fillna(0).astype(int)

#Drop the column
df_sample.drop(columns=["marketing_strategy"], inplace=True)

## subject - BERT embedding TBD

For a chunk of size 1e5

50% off absolutely everything + free shipping                                  3522

<#include "cms://messagelibrary/email/59251982/DynamicSubjectLine.itl.txt">    3306

<#include "cms://messagelibrary/email/59253142/DynamicSubjectLine.itl.txt">    3213

ONE 👏 DAY 👏 ONLY 👏                                                             2808

<#include "cms://messagelibrary/email/59252822/DynamicSubjectLine.itl.txt">    2403
                                                                               ... 

<#include "cms://messagelibrary/email/59155022/DynamicSubjectLine.itl.txt">       1

Spend this $20 or lose this $20                                                   1

What you should know about our fits                                               1

<#include "cms://messagelibrary/email/59103642/DynamicSubjectLine.itl.txt">       1

<#include "cms://messagelibrary/email/59219282/DynamicSubjectLine.itl.txt">       1

Name: subject, Length: 178, dtype: int64

In [47]:
#Initialize the BERT tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
model.eval()

def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]
    
    #Stack tensors up to give us a proper Pytorch tensor to work with
    token_embeddings = torch.stack(hidden_states, dim=0)
    
    #Remove the batch-dimension because we have only a single sentence
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    #Swapping the layer dimension with the token dimension
    token_embeddings = token_embeddings.permute(1,0,2)

    #For each token, compute average across 1st dimension. Then, average across 0th dimension.
    sentence_embedding = torch.mean(token_embeddings, dim=(0, 1))

    return sentence_embedding

In [57]:
#Try 3 sentences:
#Sentence 1) "We bet we have the fit for you"
#Sentence 2) "We’ve got your size"
#Sentence 3) "Long-weekend savings all week long"
#Sentence 4) "Get 50% off— ahead of everyone else"

s1 = "Find pants that fit perfectly"
s2 = "We bet we have the fit for you"
s3 = "50% off absolutely everything + free shipping"
s4 = "Get 50% off— ahead of everyone else"

_, token_ids_s1, segment_ids_s1 = bert_text_preparation(s1, tokenizer)
_, token_ids_s2, segment_ids_s2 = bert_text_preparation(s2, tokenizer)
_, token_ids_s3, segment_ids_s3 = bert_text_preparation(s3, tokenizer)
_, token_ids_s4, segment_ids_s4 = bert_text_preparation(s4, tokenizer)

s1_embedding = get_bert_embeddings(token_ids_s1, segment_ids_s1, model)
s2_embedding = get_bert_embeddings(token_ids_s2, segment_ids_s2, model)
s3_embedding = get_bert_embeddings(token_ids_s3, segment_ids_s3, model)
s4_embedding = get_bert_embeddings(token_ids_s4, segment_ids_s4, model)

In [58]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the S1 and S2 (similar - fit-based)
s1s2sim = 1 - cosine(s1_embedding, s2_embedding)
print(f"[INFO] Computing cosing similarity between {s1} and {s2}: {s1s2sim}")

# Calculate the cosine similarity between the S3 and S4 (similar - savings-based)
s3s4sim = 1 - cosine(s3_embedding, s4_embedding)
print(f"[INFO] Computing cosing similarity between {s3} and {s4}: {s3s4sim}")

# Calculate the cosine similarity between the S1 and S3 (dissimilar - fit vs savings)
s1s3sim = 1 - cosine(s1_embedding, s3_embedding)
print(f"[INFO] Computing cosing similarity between {s1} and {s3}: {s1s3sim}")

# Calculate the cosine similarity between the S2 and S4 (dissimilar - fit vs savings)
s2s4sim = 1 - cosine(s2_embedding, s4_embedding)
print(f"[INFO] Computing cosing similarity between {s2} and {s4}: {s2s4sim}")

[INFO] Computing cosing similarity between Find pants that fit perfectly and We bet we have the fit for you: 0.7733584046363831
[INFO] Computing cosing similarity between 50% off absolutely everything + free shipping and Get 50% off— ahead of everyone else: 0.8531660437583923
[INFO] Computing cosing similarity between Find pants that fit perfectly and 50% off absolutely everything + free shipping: 0.6893360614776611
[INFO] Computing cosing similarity between We bet we have the fit for you and Get 50% off— ahead of everyone else: 0.7552185654640198


In [43]:
#Explore the hidden_states more clearly
print ("Number of layers:", len(hidden_states))

#Pick any 1 layer
layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))

#Pick any 1 batch
batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))

#Pick any 1 token
token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)
print(f"Stacking all the hidden units...Pytorch tensor of shape {token_embeddings.size()} is created")

#Remove the batch-dimension because we have only a single sentence
token_embeddings = torch.squeeze(token_embeddings, dim=1)
print(f"Removing batch dimension...Pytorch tensor of shape {token_embeddings.size()} is created")

#Swapping the layer dimension with the token dimension
token_embeddings = token_embeddings.permute(1,0,2)
print(f"Swapping layer with token...Pytorch tensor of shape {token_embeddings.size()} is created")

#For each token, compute average across 1st dimension. Then, average across 0th dimension.
token_vecs = torch.mean(token_embeddings, dim=(0, 1))
print(f"The embedding for - Sample text - is of size {token_vecs.size()}")

Number of layers: 12
Number of batches: 1
Number of tokens: 4
Number of hidden units: 768
Stacking all the hidden units...Pytorch tensor of shape torch.Size([12, 1, 4, 768]) is created
Removing batch dimension...Pytorch tensor of shape torch.Size([12, 4, 768]) is created
Swapping layer with token...Pytorch tensor of shape torch.Size([4, 12, 768]) is created
The embedding for - Sample text - is of size torch.Size([768])


In [11]:
"""
Column - subject

1) Use BERT to convert this into a sentence vector?
2) Check for specific words of interest in the subject?
3) Other handcrafted features like LogReg or NB?
"""
if control_variable["subject"]:
  df_sample.drop(columns=["subject"], inplace=True)  

## campaign_type

for chunk of size 1e5:

20191129_US_Dockers_Dedicated_BF_50Off_FSFR_AM_Promo             3522

DOC_WelNoPurchT1_Trigger_20191226                                3306

DOC_WelNoPurchT4_Trigger_20191226                                3213

20191202_US_Dockers_Dedicated_CyberMonday_50off_FSFR_AM_Promo    2808

DOC_WelNoPurchT2_Trigger_20191226                                2403
                                                                 ... 
20190912_EU_Dockers_Mkt_360Tops_360ChinoStripe                      1

20190816_US_Dockers_Apology                                         1

20190927_EU_Dockers_Promo_FriendsFamily                             1

20191030_EU_Dockers_Mkt_FitGuide                                    1

20190919_EU_Dockers_Mkt_NewArrivals                                 1

Name: campaign_type, Length: 195, dtype: int64

Occurrences of Brand :4525

Occurrences of Collection :0

Occurrences of Core :15856

Occurrences of Dedicated :32435

Occurrences of InnovationSpotlight :1856

Occurrences of NewArrivals :2075

Occurrences of Other :0

Occurrences of ProductSpotlight :1677

Occurrences of Replen :0

Occurrences of Tops :3130

Occurrences of Trend :12996

In [12]:
"""
Column Name - campaign_type

1) Can get the old campaign_type categorical columns from this
2) Can eliminate EU entries. If you try matching US and inverting the mask, you
end up eliminating 90% of entries. So, pick EU and eliminate. There are also NA 
(typically connected to Triggers) and non-pattern conforming entries
3) Discounts can be extracted!
"""

#Find out how many different ways words like US show up in campaign_type
country_mask = df_sample["campaign_type"].str.contains(r"_[A-Z][A-Z]_", regex=True)
print(f"There are {len(df_sample[country_mask])} entries that satisfy _[A-Z][A-Z]_ regex match\n")

#Display how the discounts are presented
discount_pattern = re.compile(r"_[1-9][0-9]off_", flags = re.I)
discount_mask = df_sample["campaign_type"].str.contains(discount_pattern, regex=True)
print(f"There are {len(df_sample[discount_mask])} entries that satisfy discount regex match\n")
print("Some of them include:")
print((df_sample[discount_mask]).campaign_type.unique()[:5])

if not(control_variable["campaign_type"]):

  #Eliminate EU rows
  """
  The approach below does not work as 1e5 rows are ↓ to 78K+ rows
  
  US_mask = df_sample.campaign_type.str.contains('_US_')
  df_sample = df_sample[US_mask]
  """
  EU_mask = df_sample.campaign_type.str.contains('_EU_')
  df_sample = df_sample[~EU_mask]

  #Convert a single column into one-hot encoded campaign_type columns
  df_sample["campaign_Brand"] = df_sample.campaign_type.str.contains("Brand").astype(int)
  df_sample["campaign_Collection"] = df_sample.campaign_type.str.contains("Collection").astype(int)
  df_sample["campaign_Core"] = df_sample.campaign_type.str.contains("Core").astype(int)
  df_sample["campaign_Dedicated"] = df_sample.campaign_type.str.contains("Dedicated").astype(int)
  df_sample["campaign_InnovationSpotlight"] = df_sample.campaign_type.str.contains("InnovationSpotlight").astype(int)
  df_sample["campaign_NewArrivals"] = df_sample.campaign_type.str.contains("NewArrivals").astype(int)
  df_sample["campaign_ProductSpotlight"] = df_sample.campaign_type.str.contains("ProductSpotlight").astype(int)
  df_sample["campaign_Replen"] = df_sample.campaign_type.str.contains("Replen").astype(int)
  df_sample["campaign_Tops"] = df_sample.campaign_type.str.contains("Tops").astype(int)
  df_sample["campaign_Trend"] = df_sample.campaign_type.str.contains("Trend").astype(int)
  
  df_sample["campaign_Other"] = 1 - df_sample["campaign_Brand"] - df_sample["campaign_Collection"] - df_sample["campaign_Core"] - df_sample["campaign_Dedicated"] - df_sample["campaign_InnovationSpotlight"] - df_sample["campaign_NewArrivals"] - df_sample["campaign_ProductSpotlight"] - df_sample["campaign_Replen"] - df_sample["campaign_Tops"] - df_sample["campaign_Trend"]

  #Create discount columns
  df_sample["discount"] = 0
  df_sample.loc[discount_mask, "discount"] = (df_sample[discount_mask])["campaign_type"].str.findall(r"(?<=_)\d{2}").str.get(0).astype(int)


#Drop the campaign_type column 
df_sample.drop(columns=["campaign_type"], inplace=True)

There are 85202 entries that satisfy _[A-Z][A-Z]_ regex match

There are 30368 entries that satisfy discount regex match

Some of them include:
['20190811_US_Dockers_Tops_GraphicTees_30off_FS_Promo_SL1'
 '20191129_US_Dockers_Dedicated_BF_50Off_FSFR_AM_Promo'
 '20191229_US_Dockers_Dedicated_EOSS_50off_Promo'
 '20191021_US_Dockers_Dedicated_MSS_50off_Starts_Promo'
 '20191125_US_Dockers_Dedicated_EarlyAccess_BF_50Off_FSFR_Promo']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## offer_category

In [13]:
"""
Offer_category Columns - "offer_category"

NOTE
----
1) This contains additional info about whether it's promotion, sale, Tops etc.
that are somehow missed by the other columns. We can leverage this?

e.g. "Mens_Pants...", "Mens_ShirtTops...", "Mens_BigAndTall...", "Mens_Shorts...",
"Mens_SweatersJackets...", "Mens_Accessories...", "Promotion", "Sale", "Brand"

2) HORRIBLY BIASED (at least this column is) DATASET because (A) not even a single mention
of womens products along the lines of Mens_Shorts etc. and (B) very few entries
3) But at the end of the day - only 8.3% are filled. Hence, not using.
"""

#Extract the column as a Series + convert to str
offer_cat_series = df_sample["offer_category"].astype(str)

#Print the unique values corresponding to Mens clothing
mens_clothing_mask = offer_cat_series.str.contains("Mens")
off_cat_mens = offer_cat_series[mens_clothing_mask]
print(f"[INFO] The number of entries with Mens: {len(off_cat_mens)}")
print(f"[INFO] Printing out a few of them: {off_cat_mens.unique()[:10]}")

womens_clothing_mask = offer_cat_series.str.contains("Wom")
off_cat_womens = offer_cat_series[womens_clothing_mask]
print(f"\n[INFO] The number of entries with Womens: {len(off_cat_womens)}")
print(f"[INFO] Printing out a few of them: {off_cat_womens.unique()[:-1]}")

if control_variable['offer_cat']:
  df_sample.drop(columns=["offer_category"], inplace=True)

[INFO] The number of entries with Mens: 13351
[INFO] Printing out a few of them: ['Mens_Pants_Straight' 'Mens, Sale' 'Promotion, Mens_ShirtTops, Brand'
 'Mens' 'Mens_Pants_Relaxed' 'Promotion, Mens_Pants_Relaxed, Mens'
 'Mens_BigAndTall' 'Mens_BigandTall'
 'Mens_Pants_Relaxed, Mens_Pants_Straight' 'Mens_Pants_SlimTapered']

[INFO] The number of entries with Womens: 30
[INFO] Printing out a few of them: ['Womens' 'Sale, Womens' 'Mens, Mens_BigandTall, Womens' 'Womens, Sale'
 'Womens, Promotion'
 'Sale, Womens, Mens, Brand, Mens_ShirtTops, Mens_Accessories, Mens_SweatersJackets'
 'Mens_SweatersJackets, Mens_Accessories, Sale, Mens, Womens, Mens_ShirtTops'
 'Mens, Womens, Sale']


## Acquisiton Columns

In [14]:
"""
Acquisiton Columns - "aq_dt", "aq_mo", "aq_dow", "aq_period"

NOTE
----
1) Older customers who have not subscribed — exploit strategy should work better
2) aq_dt has negative numbers - Why?
"""
if control_variable["aq"]:
  col_drop_aq = ["aq_dt", "aq_mo", "aq_dow", "aq_period"]
  df_sample.drop(columns=col_drop_aq, inplace=True)

## Website Activity

In [15]:
"""
Website Data - "wk1_vis", "wk4_vis", "mo3_vis", "mo6_vis", "wk1_pv",
"wk4_pv", "mo3_pv", "mo6_pv", "wk1_secs", "wk4_secs", "mo3_secs", "mo6_secs"

NOTE
----
1) Extremely sparse - mostly filled with 0s
2) Marko had said website and email integration has not taken place
3) Naturally, this integration will happen in the future. Let the future team take care of it then.
"""
if control_variable["site"]:
  col_drop_site = ["wk1_vis", "wk4_vis", "mo3_vis", "mo6_vis", "wk1_pv",
"wk4_pv", "mo3_pv", "mo6_pv", "wk1_secs", "wk4_secs", "mo3_secs", "mo6_secs"]
  df_sample.drop(columns=col_drop_site, inplace=True)

## Last Time Data

In [16]:
"""
The Last Time - "last_order", "last_click", "last_cart","last_visit","last_ded_open"

NOTE
----
1) Moooostly NaN values currently
2) However, when it is not NaN - especially for last_order - it could be a really useful signal
"""
if control_variable["last"]:
  col_drop_last = ["last_order", "last_click", "last_cart","last_visit","last_ded_open"]
  df_sample.drop(columns=col_drop_last, inplace=True)

##Promotion/campaign IDs

In [17]:
"""
Promotion IDs - "offer_signature_id", "launch_id", "campaign_id"

NOTE
----
1) A bunch of strings - might be useful for either cross-feature interactions or 
campaign/promotion embeddings
2) I feel it is a nice-to-have thing, as opposed to a need-to-have-thing
"""
if control_variable["promo_ids"]:
  col_drop_promo_ids = ["offer_signature_id", "launch_id", "campaign_id"]
  df_sample.drop(columns=col_drop_promo_ids, inplace=True)

##Dtld User-Email Interactions

In [18]:
"""
Detailed user-email interaction COUNTS - "open_cnt", "click_cnt" 
""
NOTE
----
1) We already have the "opened" signal - can it be boosted further with open_cnt and click_cnt?
2) But only 28.2% and 8.3% of these counts are available
"""
if control_variable["email_cnts"]:
  col_drop_email_cnts = ["open_cnt", "click_cnt"]
  df_sample.drop(columns=col_drop_email_cnts, inplace=True)

In [19]:
"""
Detailed user-email interaction TIMES - "first_open_time", "last_open_time", "hrs_to_first_open", 
"hrs_to_last_open", "first_click_time", "last_click_time", "hrs_to_first_click", "hrs_to_last_click"

NOTE
----
1) Hmmmm...
"""
if control_variable["email_times"]:
  col_drop_email_times = ["first_open_time", "last_open_time", "hrs_to_first_open", 
"hrs_to_last_open", "first_click_time", "last_click_time", "hrs_to_first_click", "hrs_to_last_click"]
  df_sample.drop(columns=col_drop_email_times, inplace=True)

##Opt-out

In [20]:
"""
Opt Out - "first_opt_out_time", "opt_out_cnt", "last_opt_out_time", "opt_out_source", 
"opt_out_reason", "hrs_to_first_opt_out", "hrs_to_last_opt_out"

NOTE
-----
1) Marko's response: so I think there's a couple of different levels you can opt out on, just promotional emails, all emails etc.
so they probably resubscribed if they got added in later
2) I think the emailer system has checks and balances to see whether people who opt out,
should be sent mails to. But they don't keep track of the past - i.e., when someone opts out,
then resubscribes - he/she does not get counted as an opt out
3) Only 0.3% of entries have opt out filled - prev_opt_outs is 0 for 94% of entries
"""
if control_variable["opt_out"]:
  col_drop_opt_out = ["prev_optouts", "first_opt_out_time", "opt_out_cnt", "last_opt_out_time", "opt_out_source", 
"opt_out_reason", "hrs_to_first_opt_out", "hrs_to_last_opt_out"]
  df_sample.drop(columns=col_drop_opt_out, inplace=True)

#Printing the head to see

In [46]:
#Print the head of the sample dataframe now
df_sample.head(60)

,riid,opened,unsub,rev_3dv2,sends_since_last_open,message_size,subject,retention_score,recency_score,sent_time,email_type,promo,sale,campaign_Brand,campaign_Collection,campaign_Core,campaign_Dedicated,campaign_InnovationSpotlight,campaign_NewArrivals,campaign_ProductSpotlight,campaign_Replen,campaign_Tops,campaign_Trend,campaign_Other,discount
0,251392782,1,0,48.98000,16,149211,It’s almost the end,1.75000,0.00000,2019-08-11 17:11:15,1,1,0,0,0,0,0,0,0,0,0,1,0,0,30
1,273890682,0,0,27.74000,0,111097,"<#include ""cms://messagelibrary/email/59253142...",28.00000,0.00000,2020-03-14 19:43:46,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,273890682,0,0,27.74000,0,175946,"<#include ""cms://messagelibrary/email/59253042...",28.00000,0.00000,2020-03-12 18:43:46,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,273890682,1,0,27.74000,0,163449,"<#include ""cms://messagelibrary/email/59251982...",28.00000,0.00000,2020-03-07 18:20:54,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,273890682,0,0,27.74000,0,228854,"<#include ""cms://messagelibrary/email/59252822...",28.00000,0.00000,2020-03-10 17:43:46,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,260343962,1,0,59.98000,53,163400,50% off absolutely everything + free shipping,0.52830,0.00000,2019-11-29 13:07:15,1,1,0,0,0,0,1,0,0,0,0,0,0,0,50
6,206326602,1,0,39.00000,0,164971,50% off absolutely everything + free shipping,28.00000,2.28571,2019-11-29 13:08:27,1,1,0,0,0,0,1,0,0,0,0,0,0,0,50
7,270348662,1,0,74.40000,0,161567,"<#include ""cms://messagelibrary/email/59194602...",28.00000,0.00000,2019-11-11 06:17:02,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,261174382,1,0,100.80000,0,102551,Free shipping + free returns,28.00000,2.28571,2019-12-14 15:01:39,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0
9,192864422,1,0,64.79000,0,129591,These should get you through the winter,28.00000,6.85714,2019-11-21 22:06:35,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [22]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99965 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   riid                          99965 non-null  int64         
 1   opened                        99965 non-null  int64         
 2   unsub                         99965 non-null  int64         
 3   rev_3dv2                      99965 non-null  float64       
 4   sends_since_last_open         99965 non-null  int64         
 5   message_size                  99965 non-null  int64         
 6   subject                       99965 non-null  object        
 7   retention_score               99965 non-null  float64       
 8   recency_score                 99965 non-null  float64       
 9   sent_time                     99965 non-null  datetime64[ns]
 10  email_type                    99965 non-null  int64         
 11  promo                       

In [23]:
cnt = 0
reader = pd.read_csv(file_name_l[0], sep='\t', header=None, names=col_names, chunksize=1e5)